<a href="https://colab.research.google.com/github/autinn/deep-tech-talent/blob/main/Lab16_N_07_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab16: Deep Learning Assessment**

Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started!

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="https://raw.githubusercontent.com/skywong1230/IT524031Q/main/fruits.png?raw=1" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at Lab17 where we implemented transfer learning.

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


## Freeze Base Model

Next, we suggest freezing the base model, as done in Lab17. This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [ ]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. Lab17 can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224,224,3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(units=6,activation='softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs,outputs)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at Lab14 and Lab17 for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen =ImageDataGenerator(rotation_range=10,# randomly rotate images in the range (degrees, 0 to 180)
zoom_range=0.1,# Randomly zoom image
width_shift_range=0.1,# randomly shift images horizontally (fraction of total width)
height_shift_range=0.1,# randomly shift images vertically (fraction of total height)
horizontal_flip=True,# randomly flip images horizontally
vertical_flip=False,# Don't randomly flip images vertically
)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out Lab17.

In [ ]:
# Connect to Googel Drive to get the data
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/data/fruits/train/',target_size=(224,224),
                                       color_mode='rgb',
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/data/fruits/valid/',target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
36/36 [==============================] - 333s 8s/step - loss: 4.8582 - accuracy: 0.3371 - val_loss: 1.5255 - val_accuracy: 0.6231
Epoch 2/20
36/36 [==============================] - 24s 647ms/step - loss: 0.9457 - accuracy: 0.7340 - val_loss: 0.6818 - val_accuracy: 0.7994
Epoch 3/20
36/36 [==============================] - 24s 647ms/step - loss: 0.4385 - accuracy: 0.8652 - val_loss: 0.4360 - val_accuracy: 0.8754
Epoch 4/20
36/36 [==============================] - 24s 649ms/step - loss: 0.2715 - accuracy: 0.9046 - val_loss: 0.3835 - val_accuracy: 0.8875
Epoch 5/20
36/36 [==============================] - 24s 650ms/step - loss: 0.2048 - accuracy: 0.9359 - val_loss: 0.4156 - val_accuracy: 0.8784
Epoch 6/20
36/36 [==============================] - 24s 650ms/step - loss: 0.1412 - accuracy: 0.9536 - val_loss: 0.3377 - val_accuracy: 0.8967
Epoch 7/20
36/36 [==============================] - 24s 651ms/step - loss: 0.1038 - accuracy: 0.9675 - val_loss: 0.2826 - val_accuracy: 0.9271
E

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
36/36 [==============================] - 38s 878ms/step - loss: 0.1153 - accuracy: 0.9740 - val_loss: 0.1538 - val_accuracy: 0.9574
Epoch 2/20
36/36 [==============================] - 28s 756ms/step - loss: 0.0375 - accuracy: 0.9890 - val_loss: 0.2960 - val_accuracy: 0.9483
Epoch 3/20
36/36 [==============================] - 28s 760ms/step - loss: 0.0242 - accuracy: 0.9899 - val_loss: 0.1720 - val_accuracy: 0.9726
Epoch 4/20
36/36 [==============================] - 28s 757ms/step - loss: 0.0333 - accuracy: 0.9928 - val_loss: 0.1832 - val_accuracy: 0.9635
Epoch 5/20
36/36 [==============================] - 28s 762ms/step - loss: 0.0113 - accuracy: 0.9963 - val_loss: 0.2368 - val_accuracy: 0.9574
Epoch 6/20
36/36 [==============================] - 28s 756ms/step - loss: 0.0342 - accuracy: 0.9840 - val_loss: 0.1630 - val_accuracy: 0.9696
Epoch 7/20
36/36 [==============================] - 28s 758ms/step - loss: 0.0097 - accuracy: 0.9971 - val_loss: 0.2698 - val_accuracy: 0.9605

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation.

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`.

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

10/10 [==============================] - 5s 489ms/step - loss: 0.2482 - accuracy: 0.9696


[0.24820254743099213, 0.9696048498153687]